In [144]:
import numpy as np

with open('input.txt') as f:
    lines = f.read().splitlines()

map = np.array([[el for el in line] for line in lines])
map

array([['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.'],
       ['.', '#', '.', ..., '.', '.', '.'],
       ...,
       ['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.'],
       ['#', '.', '.', ..., '.', '.', '.']], dtype='<U1')

# Part 1

In [145]:
class State:
    def __init__(self, map):
        self.visited_positions = []
        self.map = map
        self.down, self.right = self.find_initial_position()
        self.direction = 'up'

    def find_initial_position(self):
        initial_position = np.where(self.map == '^')
        return initial_position[0][0], initial_position[1][0]

    def move(self):
        if self.direction == 'up':
            return self.down - 1, self.right
        elif self.direction == 'down':
            return self.down + 1 , self.right
        elif self.direction == 'left':
            return self.down, self.right - 1
        elif self.direction == 'right':
            return self.down, self.right + 1

    def update(self):
        self.next_down, self.next_right = self.move()
        if self.check_out_of_bounds():
            return self.visited_positions
        elif self.hit_obstacle():
            self.turn()
            self.next_down, self.next_right = self.move()
            self.update_position()
        else:
            self.update_position()

    def update_position(self):
        self.down = self.next_down
        self.right = self.next_right
        self.visited_positions.append((self.down, self.right))

    def check_out_of_bounds(self):
        if (
            (self.next_down < 0 or self.next_down >= self.map.shape[0]) | 
            (self.next_right < 0 or self.next_right >= self.map.shape[1])
        ):
             return True
        else:
            return False
        
    def hit_obstacle(self):
        if self.map[self.next_down, self.next_right] == '#':
            return True
        else:
            return False
        
    def turn(self):
        switch_direction ={
            'up': 'right',
            'right': 'down',
            'down': 'left',
            'left': 'up'
        }
        self.direction = switch_direction[self.direction]


state = State(map)

while True:
    # print(state.distance)
    if state.update():
        break

len(set(state.visited_positions))

4752

# Part 2

In [146]:
# create new maps

new_maps = []
visited_positions = set(state.visited_positions)
for i in range(map.shape[0]):
    for j in range(map.shape[1]):
        if (i, j) not in visited_positions:
            continue
        if map[i, j] in '#^':
            continue
        else:
            new_map = map.copy()
            new_map[i, j] = 'O'
            new_maps.append(new_map)


class NewState:
    def __init__(self, map):
        self.visited_positions = set()
        self.map = map
        self.down, self.right = self.find_initial_position()
        self.direction = 'up'

    def find_initial_position(self):
        initial_position = np.where(self.map == '^')
        return initial_position[0][0], initial_position[1][0]
    
    def find_next_position(self):
        self.next_down, self.next_right = self.move()

        if self.check_out_of_bounds():
            return 'out_of_bounds'
        
        if self.hit_obstacle():
            self.turn()
            self.find_next_position()

    def update(self):
        if self.find_next_position() == 'out_of_bounds':
            return 'out_of_bounds'
        
        self.update_position()
        
        if self.check_loop():
            return 'loop'
        
        self.visited_positions.add((self.down, self.right, self.direction))
        

    def update_old(self):
        self.next_down, self.next_right = self.move()
        
        if self.check_out_of_bounds():
            return 'out_of_bounds'
        
        if self.hit_obstacle():
            self.turn()
            self.update()

        self.update_position()
        
        self.visited_positions.add((self.down, self.right, self.direction))
        
        

    def move(self):
        if self.direction == 'up':
            return self.down - 1, self.right
        elif self.direction == 'down':
            return self.down + 1 , self.right
        elif self.direction == 'left':
            return self.down, self.right - 1
        elif self.direction == 'right':
            return self.down, self.right + 1

    def check_loop(self):
        if (self.down, self.right, self.direction) in self.visited_positions:
            return True
        else:
            return False

    def update_position(self):
        self.down = self.next_down
        self.right = self.next_right
        
    def check_out_of_bounds(self):
        if (
            (self.next_down < 0) or 
            (self.next_down >= self.map.shape[0]) or 
            (self.next_right < 0) or 
            (self.next_right >= self.map.shape[1])
        ):
            return True
        else:
            return False
        
    def hit_obstacle(self):
        if self.map[self.next_down, self.next_right] in '#O':
            return True
        else:
            return False
        
    def turn(self):
        switch_direction = {
            'up': 'right',
            'right': 'down',
            'down': 'left',
            'left': 'up'
        }
        self.direction = switch_direction[self.direction]   

loops = []
out_of_bounds = 0
visited_positions = []
for i, new_map in enumerate(new_maps):
    if i % 500 == 0:
        print(i)

    new_state = NewState(new_map)
    while True:
        if new_state.update() == 'loop':
            loops.append(new_map)
            visited_positions.append(new_state.visited_positions)
            break
        if new_state.update() == 'out_of_bounds':
            out_of_bounds += 1
            break

len(loops), out_of_bounds, len(new_maps)


0
500
1000
1500
2000
2500
3000
3500
4000
4500


(1719, 3032, 4751)

In [143]:
replace_direction = {
    'up': '|',
    'down': '|',
    'left': '-',
    'right': '-'
}


n = 5
map = loops[n]
for state in visited_positions[n]:
    *pos, direction = state
    map[pos[0], pos[1]] = replace_direction[direction]


["".join(l) for l in map.tolist()]

['....#.......#.............#..##...................#..#..#..................................#....#.................................',
 '............|-----------------O.#..#.............##.....#....#....................#.............................................#.',
 '.#....#.....|.............|..|...............................|----------------------------------#.#...............................',
 '.......#....|..#..........|..|....#........#.................|................#...............#|..........#.......................',
 '..#....|----|------------#---|.#...........#.................|.................................|......................#...........',
 '.#.....|....|..#........|....#.|--#...............#..#.......|.................................|......#..........#................',
 '.#.....|....|...........|......|.||-----------#........#.....|.................................|#...............................#.',
 '.......|....|..#........|..#...|#||..........|.......